# Using Genecodr to visualize gene sets

##### Author: Damon Pham

This demo takes in a gmt file, and embeds each of its gene sets into the latent space of a variational autoencoder (VAE). The VAE has been trained on 700k+ gene sets from Enrichr and ChEA3 libraries, and user-submitted Enrichr queries. The clustering of the gene sets within the latent space is visualized via Clustergrammer.

# Imports

In [ ]:
import os
import numpy as np
import scipy as sp
import pandas as pd

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
all_genes = pd.read_csv('../models/gene_name_conversion.tsv', sep='\t', index_col=0)

Additional requirements (from imported scripts):
* csv
* itertools
* pickle
* h5py
* h5sparse
* scipy
* keras

In [ ]:
os.chdir('../scripts')
from gvm_scripts import *
from vae_scripts import *
os.chdir('../notebooks')

### VAE weights

See the README for a link to download weights for the best VAE model.

In [ ]:
vae_fname = '../models/vae_weights_1500m_100l.h5'
assert os.path.exists(vae_fname)

# Format gmt

### Convert the gmt to a gene vector matrix (gvm)

Your gmt must:
* separate entries within a geneset by tabs, and separate genesets by newlines.
* have sample names in the first entry of a row only (before the first tab). Anything after the first tab is interpreted as a gene. 
* have no empty rows, and have each row include a sample name and at least one gene.

The demo file is the ARCHS4 Tissues gmt from Enrichr. 
To use your own gmt, substitute its file path into the line below.

In [ ]:
### YOUR GMT FILE ############
gmt_fname = '../data/demo.txt'
##############################

In [ ]:
lib_name = os.path.splitext(gmt_fname.rsplit('/', 1)[-1])[0]
gvm_fname = '../data/' + lib_name + '.h5'
formatted_gvm_fname = '../data/' + lib_name + '_FORMATTED.h5'

In [ ]:
if os.path.isfile(gvm_fname): 
    gvm = open_gvm(gvm_fname)
else:
    gvm = convert_genesetlist(get_genesetlist(gmt_fname, 'gmt_fname'), 
                              to='gvm_h5',
                              output_fname=gvm_fname)

The gvm object is a dictionary with the data in 'gvm', row-index in 'idx', and column-index in 'col'.

Each row vector is a geneset. The number of rows is the number of samples; the number of columns is the total number of genes.

In [ ]:
print('Pre-formatting, the gvm has %d rows, or samples/genesets, and %d columns, or genes.'%gvm['gvm'].shape)

In [ ]:
print('First five samples:')
gvm['idx'][:5]

In [ ]:
print('First five genes:')
gvm['col'][:5]

### Format the gvm's gene column-index to match that of the matrix used to train the autoencoder

Formatting will:
* __be skipped, if a previously-made formatted gvm exists.__
* capitalize gene names.
* remove gene name suffixes, if > 10% of gene names have commas or underscores. (e.g. 'AATF,1.0' --> 'AATF).
* convert gene names to HUGO identifiers.
* discard unconvertible gene names.
* discard "rare" genes: genes not included in the ~20,000 used to train the VAE.
* take the union for genes mapped onto the same HUGO identifier.
* __drop samples which have less than `min_gs_size` genes, or have lost more than `max_gs_loss` of their genes.__
* re-label the gene index with numerical gene IDs (gene names can be recovered with `gene_name_conversion.tsv`).
* re-order the column and row indices.
* __save the new gvm to `formatted_gvm_fname`__

Modify the below chunk to change the bolded actions.

In [ ]:
summary = format_gvm_h5(gvm_fname = gvm_fname, 
                        all_genes = all_genes,
                        output_fname = formatted_gvm_fname, # <-- desired output file name
                        max_gs_loss=1.0, # <-- samples which have lost a greater proportion of genes are removed.
                        min_gs_size=1, # <-- samples which become smaller than this are removed.
                        overwrite = True) # <-- should `output_fname` be overwritten, if it exists?

In [ ]:
n_labels, n_genes = get_gvm_size(formatted_gvm_fname)
print('After formatting, the gvm has %d rows, or samples/genesets, and %d columns, or genes.'%get_gvm_size(formatted_gvm_fname))
print('(Columns for genes not present in the gmt will be emtpy, and are necessary for padding the vectors.)')

# Get Latent Space Embedding

### Construct autoencoder

See the README for a link to download weights for the best VAE model.

The backup weights included in this repo are less-optimal, but within Github's file size limits.

In [ ]:
m, l = 1500, 100
model = build_vae(input_dim=n_genes, middle_dim = m, latent_dim = l, variational=True)
vae, enc, dec = (model['vae'], model['enc'], model['dec'])
vae.load_weights(vae_fname)            

### Encode genesets

In [ ]:
z = enc.predict_generator(
    GeneVec_Generator(formatted_gvm_fname, gvm_path='gvm', batch_size=1000, shuffle=False),
    workers=4, use_multiprocessing=True, verbose=0)
z.shape

# Compute Proximity Matrices

The cosine distance has been shown to perform better on an enrichment benchmark. Thus, this demo uses the cosine distance to perform clustering. Euclidean distance is computed below for completion.

### Euclidean distance

In [ ]:
euc_dist = pairwise_distances(z, metric='euclidean')

In [ ]:
np.min(euc_dist), np.max(euc_dist)

### Cosine similarity

In [ ]:
cos_sim = cosine_similarity(z)

In [ ]:
np.min(cos_sim), np.max(cos_sim)

### Save results to pd.DataFrame

In [ ]:
labels = open_gvm(formatted_gvm_fname)['idx']

euc_dist_df = pd.DataFrame(euc_dist, index=labels, columns=labels)
cos_sim_df = pd.DataFrame(cos_sim, index=labels, columns=labels) 

In [ ]:
euc_dist_df.iloc[:5, :5]

In [ ]:
cos_sim_df.iloc[:5, :5]

### Demo for saving & loading results

In [ ]:
euc_dist_df.to_pickle('../data/%s_DIST_EUC.pkl'%lib_name)
cos_sim_df.to_pickle('../data/%s_DIST_COS.pkl'%lib_name)

# could also use:
# cos_sim_df.to_csv('COS_SIM_CSV_PATH.csv')

In [ ]:
cos_sim_df2 = pd.read_pickle('../data/%s_DIST_COS.pkl'%lib_name)
assert np.all(cos_sim_df == cos_sim_df2)

# Clustergrammer

In [ ]:
# import widget classes and instantiate Network instance
from clustergrammer_widget import *

### Cosine similarities

In [ ]:
net = Network(clustergrammer_widget)

# load matrix file
net.load_df(cos_sim_df)

# cluster using default parameters
net.cluster()

# make interactive widget
net.widget()